# Data Monitoring Automation

This notebook documents the code and progress as I work my way towards setting up an automated process to monitor the performance of the fraud model(s), across partners and products. 

In [22]:
# Library Imports

import pandas as pd
import numpy as np
import trellis
import os
from avant_python_utils.email import send_email
from datalaketools.connectors.presto_db import PrestoDB
presto = PrestoDB()
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc,recall_score,precision_score,accuracy_score
###

## Base Data Creation

In [2]:
#trellis.start()
# fraud = trellis.connect('us_fraud_follower')
#parent_dir_path = os.path.dirname(os.path.abspath(__file__)) - REMOVE COMMENT IN PYTHON SCRIPT
parent_dir_path = os.getcwd()
subject = 'Avant Model Monitor Weekly Report (Data Only)'
credentials = {'username': trellis.keys('automate_email')['email'], 'password': trellis.keys('automate_email')['pw']}


In [3]:
#SQL query parameters
loan_window = 'week'

Query to obtain data at a loan ID level. 

In [87]:
df_raw = presto.execute_df('''
SELECT
  l.id as loan_id
, l.loan_processing_start_time
, date_trunc('{LOAN_WINDOW}', l.loan_processing_start_time) as entered_lp_week
, l.status
, case when l.status in ('current','late','paid_off','charged_off') then 1 else 0 end as issued
, case when c.high_confidence_fraud_indicator=true or cfl.id is not null then 1 else 0 end as high_confidence_fraud_indicator
, cast(fd.score_4 as double) as score_4
, cast(fd.score_5 as double) as score_5
, coalesce(cast(fd.score_5 as double), cast(fd.score_4 as double)) as hard_score
, l.state
, l.payment_method
, l.loan_amount
, ca.product_type
, vrdt.risk_summary_identity_high
, vrdt.risk_summary_identity_medium
, vrdt.risk_summary_identity_low
FROM avant.dw.customer_applications ca
LEFT JOIN avant.dw.loans l on l.customer_application_id = ca.id
JOIN avant.dw.customers c
  ON c.id = l.customer_id
  
  -- getting fraud scores
LEFT JOIN (
  SELECT
    l.id as loan_id
  , json_extract_scalar(fd.model_scores, '$["fraud/en-US/4.1.0"]["score"]') as score_4
  , json_extract_scalar(fd.model_scores, '$["fraud/en-US/5.0.0"]["score"]') as score_5
  , fd.id as fraud_decision_id
  , row_number() over (partition by l.id order by fd.created_at desc) as row_num
  FROM avant.dw.loans l
  JOIN avant.avant_basic.fraud_decisions fd
    ON fd.customer_application_id = l.customer_application_id
    AND fd.created_at AT TIME ZONE 'America/Chicago' >= l.loan_processing_start_time
WHERE l.loan_processing_start_time > date '2019-06-30'
) fd 
  ON fd.loan_id = l.id 
  AND fd.row_num=1
  -- getting fraud indicator
LEFT JOIN avant.avant_basic.confirmed_fraud_logs cfl 
  ON cfl.customer_id = c.id
  
    -- filtering for valid loans to evaluate performance on
  -- JOIN avant.dw.loan_performance_by_installment lp 
  -- ON lp.loan_id = l.id 
  -- AND lp.installment_number = 1
  -- AND lp.installment_date <= date_add('day', -64, current_timestamp)

  
  -- adding identity tier a loan was assigned to and fraud_review flag
  LEFT JOIN avant.dw_temp_newver.verifications_risks_decisions_test vrdt
  on ca.id = vrdt.customer_application_id and vrdt.row_num_recent = 1
WHERE l.loan_processing_start_time > date '2019-06-30'
'''.format(LOAN_WINDOW = loan_window))


In [88]:
#Remove records with no fraud score
df = df_raw[df_raw.score_5.notnull()]

## Connecting to Google Sheets

In [6]:
#Importing the module
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

In [7]:
#The scope is always look like this so we did not need to change anything
scope = [
   'https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
#Name of our Service Account Key
google_key_file = 'service_key.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_key_file, scope)
gc = gspread.authorize(credentials)

In [8]:
import seaborn as sns
#I would use tips dataset as an example
tips = sns.load_dataset('tips')

In [9]:
#This is the Worksheet ID
spreadsheet_key = '14ROlpuOP9IkixM5-nn1Pc0ux6kWgmj7c62NzdDl-5hU'
#This is the sheet name
wks_name = 'test_sheet'
#We upload the tips data to our Google Sheet. Setting the row_names to False if you did not want the index to be included
d2g.upload(tips, spreadsheet_key, wks_name, credentials=credentials, row_names=False)

<Worksheet 'test_sheet' id:0>

## Model Evaluation Pipeline

In [89]:
df['prediction'] = np.where(df['score_5'] > 0.05, 1, 0)
#df['prediction'] = [1 if x > 0.05 else 0 for x in df['score_5']]
#df['prediction'] = list(np.where(df['score_5'] > 0.05, 1, 0)) 
# All three solutions raise the settingwithcopywarining

/home/jovyan/.conda-envs/gkrishna_env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [90]:
# Checking fraud stats for QC later on
df['high_confidence_fraud_indicator'].sum()

3305

In [91]:
precision_score(y_true = df['high_confidence_fraud_indicator'], y_pred = df['prediction'], pos_label = 1)

0.12580357561822533

In [92]:
recall_score(y_true = df['high_confidence_fraud_indicator'], y_pred = df['prediction'], pos_label = 1)

0.811195158850227

In [93]:
tp = df['high_confidence_fraud_indicator'] * df['prediction']
fp = (1-df['high_confidence_fraud_indicator']) * df['prediction']
fn =  df['high_confidence_fraud_indicator'] * (1-df['prediction'])
#precision
tp.sum()/(tp.sum() + fp.sum())
#recall
tp.sum()/(tp.sum() + fn.sum())

0.811195158850227

In [94]:
df[(df['score_5'] > 0.05) & (df['high_confidence_fraud_indicator'] == 1)]['loan_id'].count()

2681

In [51]:
def evaluator(dframe):
    tp = dframe['high_confidence_fraud_indicator'] * dframe['prediction']
    fp = (1-dframe['high_confidence_fraud_indicator']) * dframe['prediction']
    fn =  dframe['high_confidence_fraud_indicator'] * (1-dframe['prediction'])
    return tp.sum()/(tp.sum() + fn.sum())

In [52]:
test = df.groupby('entered_lp_week').apply(evaluator)

/home/jovyan/.conda-envs/gkrishna_env/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in long_scalars
  """


In [61]:
test = df.groupby('entered_lp_week').sum(['high_confidence_fraud_indicator'])

In [62]:
test

,loan_id,issued,high_confidence_fraud_indicator,score_4,score_5,hard_score,loan_amount,risk_summary_identity_high,risk_summary_identity_medium,risk_summary_identity_low
entered_lp_week,,,,,,,,,,
2019-09-16 00:00:00.000,5206432608,1391,2,20.260687,20.545222,20.545222,8396894.90,37,92,1255
2019-09-23 00:00:00.000,12413904422,3311,1,62.882609,53.026159,53.026159,19297925.85,102,196,3008
2019-09-30 00:00:00.000,13139923850,3496,4,88.968101,56.435366,56.435366,20875639.92,96,176,3205
2019-10-07 00:00:00.000,13454895897,3571,1,86.799867,57.195166,57.195166,21243609.03,92,174,3294
2019-10-14 00:00:00.000,11915712885,3155,2,80.168490,45.257475,45.257475,18674092.76,82,137,2921
2019-10-21 00:00:00.000,12563481203,3319,1,81.229843,49.078554,49.078554,19586549.44,126,125,3039
2019-10-28 00:00:00.000,12953046357,3414,1,84.790375,54.208076,54.208076,20293415.98,112,139,3149
2019-11-04 00:00:00.000,12501942568,3288,3,96.609019,57.369686,57.369686,19242638.11,121,155,2997
2019-11-11 00:00:00.000,11354106790,2980,2,174.372339,55.179906,55.179906,17694050.63,121,137,2712


In [54]:
sample = df.query('entered_lp_week == "2020-03-16 00:00:00.000"')

In [55]:
sample

,loan_id,loan_processing_start_time,entered_lp_week,status,issued,high_confidence_fraud_indicator,score_4,score_5,hard_score,state,payment_method,loan_amount,product_type,risk_summary_identity_high,risk_summary_identity_medium,risk_summary_identity_low,prediction
4,3923223,2020-03-19 18:00:27.856,2020-03-16 00:00:00.000,late,1,0,NaN,0.003550,0.003550,VA,ach,12000.00,installment,False,False,True,0
60,3925773,2020-03-22 21:55:30.873,2020-03-16 00:00:00.000,current,1,0,0.158139,0.006922,0.006922,MI,ach,8888.53,refinance,False,False,True,0
63,3921996,2020-03-18 10:29:05.546,2020-03-16 00:00:00.000,current,1,0,0.089295,0.005521,0.005521,IL,ach,4800.00,installment,False,False,True,0
136,3913381,2020-03-16 18:24:58.865,2020-03-16 00:00:00.000,current,1,0,0.258932,0.006109,0.006109,GA,paper_check,4200.00,installment,False,False,True,0
234,3922308,2020-03-20 09:49:43.901,2020-03-16 00:00:00.000,current,1,0,NaN,0.008517,0.008517,ME,ach,17700.00,installment,False,False,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126567,3923621,2020-03-20 10:41:05.948,2020-03-16 00:00:00.000,current,1,0,NaN,0.005851,0.005851,AL,ach,2100.00,installment,False,False,True,0
126584,3921689,2020-03-17 20:25:58.463,2020-03-16 00:00:00.000,current,1,0,NaN,0.013702,0.013702,MA,ach,6400.00,installment,True,False,False,0
126686,3924699,2020-03-21 11:13:12.370,2020-03-16 00:00:00.000,current,1,0,0.055026,0.004126,0.004126,FL,ach,9373.90,refinance,False,False,True,0
126760,3922831,2020-03-19 10:21:49.812,2020-03-16 00:00:00.000,current,1,0,NaN,0.006104,0.006104,OH,ach,5300.00,installment,False,False,True,0


In [56]:
df = sample
tp = df['high_confidence_fraud_indicator'] * df['prediction']
fp = (1-df['high_confidence_fraud_indicator']) * df['prediction']
fn =  df['high_confidence_fraud_indicator'] * (1-df['prediction'])
#precision
tp.sum()/(tp.sum() + fp.sum())
#recall
tp.sum()/(tp.sum() + fn.sum())

1.0

In [57]:
df['high_confidence_fraud_indicator'].sum()

1

In [84]:
loan_window = 'week'
df_raw = presto.execute_df('''
SELECT
  l.id as loan_id
, l.loan_processing_start_time
, date_trunc('{LOAN_WINDOW}', l.loan_processing_start_time) as entered_lp
, l.status
, case when l.status in ('current','late','paid_off','charged_off') then 1 else 0 end as issued
, case when c.high_confidence_fraud_indicator=true or cfl.id is not null then 1 else 0 end as high_confidence_fraud_indicator
, cast(fd.score_4 as double) as score_4
, cast(fd.score_5 as double) as score_5
, coalesce(cast(fd.score_5 as double), cast(fd.score_4 as double)) as hard_score
, l.state
, l.payment_method
, ca.product_type
FROM avant.dw.customer_applications ca
LEFT JOIN avant.dw.loans l on l.customer_application_id = ca.id
JOIN avant.dw.customers c
  ON c.id = l.customer_id
LEFT JOIN (
  SELECT
    l.id as loan_id
  , json_extract_scalar(fd.model_scores, '$["fraud/en-US/4.1.0"]["score"]') as score_4
  , json_extract_scalar(fd.model_scores, '$["fraud/en-US/5.0.0"]["score"]') as score_5
  , fd.id as fraud_decision_id
  , row_number() over (partition by l.id order by fd.created_at desc) as row_num
  FROM avant.dw.loans l
  JOIN avant.avant_basic.fraud_decisions fd
    ON fd.customer_application_id = l.customer_application_id
    AND fd.created_at AT TIME ZONE 'America/Chicago' >= l.loan_processing_start_time
WHERE l.loan_processing_start_time BETWEEN date_add('week', -53, current_timestamp) AND date_trunc('week',current_timestamp) 
) fd 
  ON fd.loan_id = l.id 
  AND fd.row_num=1
LEFT JOIN avant.avant_basic.confirmed_fraud_logs cfl 
  ON cfl.customer_id = c.id
WHERE l.loan_processing_start_time BETWEEN date_add('week', -53, current_timestamp) AND date_trunc('week',current_timestamp)  
'''.format(LOAN_WINDOW = loan_window))

df_perf_raw = presto.execute_df('''
SELECT
  l.id as loan_id
, case when lp.first_delinquency_30_installment_number = 1 then 1 else 0 end as first_30
, case when lp.first_delinquency_60_installment_number = 1 then 1 else 0 end as first_60
, case when lp.first_delinquency_120_installment_number = 1 then 1 else 0 end as first_120
, lp_3.balance_30 as lp3_balance_30
, lp_6.balance_30 as lp6_balance_30
, case when lp_3.loan_id is null then null else (case when lp_3.balance_30 > 10 then 1 else 0 end) end as third_30
, case when lp_3.loan_id is null then null else (case when lp_3.balance_60 > 10 then 1 else 0 end) end as third_60
, case when lp_6.loan_id is null then null else (case when lp_6.balance_30 > 10 then 1 else 0 end) end as sixth_30
, case when lp_6.loan_id is null then null else (case when lp_6.balance_60 > 10 then 1 else 0 end) end as sixth_60
, lp.first_delinquency_30_installment_number
, lp.first_delinquency_60_installment_number
, lp.first_delinquency_120_installment_number
, l.loan_amount
, l.loan_processing_start_time
, date_trunc('month', l.loan_processing_start_time) as issued_month
FROM avant.dw.loans l
JOIN avant.dw.loan_performance_by_installment lp 
  ON lp.loan_id = l.id 
  AND lp.installment_number = 1
LEFT JOIN avant.dw.loan_performance_by_installment lp_3 
  ON lp_3.loan_id = l.id 
  AND lp_3.installment_number = 3 
  AND lp_3.installment_date <= date_add('day', -64, current_timestamp)
LEFT JOIN avant.dw.loan_performance_by_installment lp_6 
  ON lp_6.loan_id = l.id 
  AND lp_6.installment_number = 6 
  AND lp_6.installment_date <= date_add('day', -64, current_timestamp)
WHERE l.loan_processing_start_time BETWEEN date_add('week', -53, current_timestamp) AND date_trunc('week',current_timestamp)  
''')




In [86]:
df['high_confidence_fraud_indicator'].sum()

38

In [85]:
#Merge previous queries
df_perf = pd.merge(df_perf_raw, df_raw, how = 'inner', on = 'loan_id')

#Remove records with no fraud score
df = df_perf[df_perf.score_5.notnull()]